In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tensorflow.python.ops import rnn, rnn_cell
from yahoo_finance import Share

In [2]:
day_len = 9
H = 256 # number of hidden layer neurons
D = 9 # input dimensionality
C = 2 # ouput classes 
n_input = 3 # MNIST data input (img shape: 2*2)
n_steps = 3 # timesteps

In [3]:
# get stock from yahoo-finance
stock = Share('2330.TW')
today = datetime.date.today()
stock_data = stock.get_historical('2014-01-01', str(today))
stock_data.reverse()
print len(stock_data)

750


In [4]:
# delete the zero-volumn data
i = 0
while( i < len(stock_data)):
    if (int(stock_data[i].get('Volume')) <= 0):
        stock_data.remove(stock_data[i])
        i = -1
    i += 1
print 'data without zero volumn', len(stock_data)

data without zero volumn 701


In [5]:
# training and testing data
data_X = np.zeros((len(stock_data)-day_len,day_len), dtype=np.float)
data_Y = np.zeros((len(stock_data)-day_len,2), dtype = np.float)
data_tag = []

for i in xrange(0, len(data_X)):
    for j in xrange(0, day_len):
        data_X[i,j] = float(stock_data[i+j].get('Close')) - float(stock_data[i].get('Close'))
        tmp = float(stock_data[i+day_len].get('Close')) - float(stock_data[i].get('Close'))
    if tmp < 0 :
        data_Y[i,0] = tmp
        data_tag.append('fall')
    else :
        data_Y[i,1] = tmp
        data_tag.append('rise')
    
len_train = int(len(data_X) * 0.8)
len_test = len(data_X) - len_train

train_X = data_X[0:len_train]
train_Y = data_Y[0:len_train]
test_X = data_X[len_train::]
test_Y = data_Y[len_train::]
train_tag = data_tag[0:len_train]
test_tag = data_tag[len_train::]

In [6]:
test_Y, test_tag

(array([[ -5. ,   0. ],
        [ -6.5,   0. ],
        [ -8. ,   0. ],
        [-10. ,   0. ],
        [ -7. ,   0. ],
        [ -9.5,   0. ],
        [ -8.5,   0. ],
        [-10. ,   0. ],
        [-11. ,   0. ],
        [ -9.5,   0. ],
        [ -7. ,   0. ],
        [ -6. ,   0. ],
        [ -3. ,   0. ],
        [  0. ,   0. ],
        [ -0.5,   0. ],
        [ -1. ,   0. ],
        [  0. ,   1. ],
        [  0. ,   7. ],
        [  0. ,   4. ],
        [  0. ,   8.5],
        [  0. ,  11. ],
        [  0. ,  12. ],
        [  0. ,  10. ],
        [  0. ,   9.5],
        [  0. ,  12. ],
        [  0. ,  10.5],
        [  0. ,   6.5],
        [  0. ,  10. ],
        [  0. ,   7.5],
        [  0. ,  10.5],
        [  0. ,   5.5],
        [  0. ,   5. ],
        [  0. ,   6.5],
        [  0. ,   1.5],
        [  0. ,   4. ],
        [  0. ,   2.5],
        [  0. ,   4.5],
        [  0. ,   4. ],
        [ -1.5,   0. ],
        [ -3. ,   0. ],
        [ -6.5,   0. ],
        [ -6. , 

In [7]:
# LSTM
# placeholder
input_x = tf.placeholder(tf.float32, [None,D], name="input_x")
input_y = tf.placeholder(tf.float32, [None,C], name="input_y")

# weights and biases
w1 = tf.get_variable("W1", shape=[H, C], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.constant(0.01, shape = [C], name = "B1"))

input_x1 = tf.reshape(input_x, [-1,n_steps, n_input])
input_x2 = tf.transpose(input_x1, [1, 0, 2])
input_x3 = tf.reshape(input_x2, [-1, n_input])
input_x4 = tf.split(0, n_steps, input_x3)

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=1.0)
outputs, states = rnn.rnn(lstm_cell, input_x4, dtype=tf.float32)
score = tf.matmul(outputs[-1], w1) + b1

loss = tf.reduce_mean(tf.reduce_sum(tf.square(score-input_y),reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss) 

In [8]:
init = tf.initialize_all_variables()

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
      
    for i in range(10000):
        # training
        sess.run(optimizer, feed_dict={input_x: train_X, input_y: train_Y})
        
        if i % 50 == 0:

            pred_tag = []
            print i
            print 'loss:', sess.run(loss, feed_dict={input_x:train_X, input_y: train_Y})

            train_prediction = sess.run(score, feed_dict={input_x: train_X})
            test_prediction = sess.run(score, feed_dict={input_x: test_X})
            for j in xrange(0, len(test_prediction)):
                #print test_prediction[j]
                if test_prediction[j,0]*(-1) - test_prediction[j,1] > 0:
                    pred_tag.append('fall')
                else:
                    pred_tag.append('rise')

0
loss: 22.7316
50
loss: 5.28851
100
loss: 3.76231
150
loss: 3.38248
200
loss: 3.10385
250
loss: 2.82961
300
loss: 2.515
350
loss: 2.1518
400
loss: 1.75575
450
loss: 1.35722
500
loss: 0.98619
550
loss: 0.660866
600
loss: 0.408688
650
loss: 0.227457
700
loss: 0.119916
750
loss: 0.0627333
800
loss: 0.0339014
850
loss: 0.0196483
900
loss: 0.0116302
950
loss: 0.00736655
1000
loss: 0.0049198
1050
loss: 0.00335978
1100
loss: 0.00388505
1150
loss: 0.00251113
1200
loss: 0.00121042
1250
loss: 0.0028737
1300
loss: 0.000672497
1350
loss: 0.00046056
1400
loss: 0.0016667
1450
loss: 0.000250597
1500
loss: 0.000175297
1550
loss: 0.000130368
1600
loss: 0.000308576
1650
loss: 0.00110099
1700
loss: 4.56278e-05
1750
loss: 0.00020873
1800
loss: 0.000100654
1850
loss: 0.00257675
1900
loss: 0.00159452
1950
loss: 1.59359e-05
2000
loss: 0.000248754
2050
loss: 0.000208175
2100
loss: 3.31071e-06
2150
loss: 1.87184e-06
2200
loss: 0.0017986
2250
loss: 1.42126e-05
2300
loss: 8.77554e-07
2350
loss: 2.15839e-05
2400

In [10]:
print pred_tag
print test_tag

['fall', 'rise', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'rise', 'fall', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'fall', 'fall', 'fall', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'fall', 'fall', 'fall', 'rise', 'fall', 'fall', 'rise', 'fall', 'rise', 'fall', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'fall',

In [12]:
# confusion matrix
tp = 0
fp = 0
fn = 0
tn = 0
for i in xrange (0, len(pred_tag)):
    if pred_tag[i] == test_tag[i] and pred_tag[i] == 'rise':
        tp = tp + 1
    elif test_tag[i] == 'fall' and pred_tag[i] == 'rise':
        fp = fp + 1
    elif test_tag[i] == 'rise' and pred_tag[i] == 'fall':
        fn = fn + 1
    else:
        tn = tn + 1
print 'Accuracy', float(tp + tn) / len(pred_tag)
print 'Precision Rate', float(tp) / float(tp + fp)
print 'Recall Rate', float(tp) / float(tp + fn)

Accuracy 0.863309352518
Precision Rate 0.920454545455
Recall Rate 0.870967741935
